In [1]:
import pandas as pd
import pickle
import numpy as np
import re 

# read xlxs
df1 = pd.read_excel('LOYALTY.VER2.xlsx', sheet_name='POS Data')
df2 = pd.read_excel('LOYALTY.VER2.xlsx', sheet_name='Loyalty')
df3 = pd.read_excel('LOYALTY.VER2.xlsx', sheet_name='Hierachy Categories & Barcodes')

# save to pickle
df1.to_pickle("posdata.pkl")
df2.to_pickle("loyalty.pkl")
df3.to_pickle("categories.pkl")

#read the pickle file
picklefile1 = open('posdata.pkl', 'rb')
picklefile2 = open('loyalty.pkl', 'rb')
picklefile3 = open('categories.pkl', 'rb')
#unpickle the dataframe
df_posdata = pickle.load(picklefile1)
df_loyalty = pickle.load(picklefile2)
df_categories = pickle.load(picklefile3)
#close file
picklefile1.close()
picklefile2.close()
picklefile3.close()


pd.set_option('display.max_rows', 65)

In [2]:
df_start = df_categories.groupby(["Category Final"]).count().sort_values("Barcode", ascending = False)
df_pc = pd.merge(df_posdata, df_categories, on='Barcode', how='inner')

#νέο df με προϊόντα ανά καλάθι
df_baskets = df_pc.groupby(["Basket_ID"]).sum("Quantity").sort_values("Quantity", ascending = False)
df_loyalty.rename(columns = {'Cardholder':'LoyaltyCard_ID'}, inplace = True)
df_all = pd.merge(df_posdata, df_loyalty, on='LoyaltyCard_ID', how='inner')
all_data = df_all = pd.merge(df_all, df_categories, on='Barcode', how='inner')
df_sumquantitybasket = df_posdata.groupby(["Basket_ID"]).sum("Quantity")

# https://stackoverflow.com/questions/43348194/pandas-select-rows-if-id-appear-several-time
dca = all_data[all_data['Basket_ID'].duplicated(keep=False)]

# Referenced: https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
dca['Grouped'] = dca.groupby('Basket_ID')['Category Final'].transform(lambda x: ','.join(x))
dcb = dca[['Basket_ID', 'Grouped']].drop_duplicates()


df_a = pd.merge(dcb,df_sumquantitybasket, on='Basket_ID', how='inner')
sum = df_sumquantitybasket.drop(['Barcode','LoyaltyCard_ID'], axis=1)
a = pd.merge(dcb, all_data, on='Basket_ID', how='inner')
a = a.groupby(['Basket_ID']).sum('Quantity')


# https://stackoverflow.com/questions/43348194/pandas-select-rows-if-id-appear-several-time
dff = all_data[all_data['Basket_ID'].duplicated(keep=False)]

# Referenced: https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
dff['Grouped'] = dff.groupby('Basket_ID')['Category Final'].transform(lambda x: ','.join(x))
d2 = dff[['Basket_ID', 'Grouped','Status','Date']].drop_duplicates()

C:\Users\Λου\AppData\Local\Temp\ipykernel_17492\3687066212.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dca['Grouped'] = dca.groupby('Basket_ID')['Category Final'].transform(lambda x: ','.join(x))
C:\Users\Λου\AppData\Local\Temp\ipykernel_17492\3687066212.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff['Grouped'] = dff.groupby('Basket_ID')['Category Final'].transform(lambda x: ','.join(x))


In [3]:
df_baskets = df_posdata.groupby(["Basket_ID"]).sum("Quantity")
df_baskets

,Barcode,Quantity,Value,LoyaltyCard_ID
Basket_ID,,,,
C117311B124D20181119,24889847425551,5,13.63,586555
C117311B140D20181027,36611051419462,8,23.17,938488
C117311B157D20181102,7454214154013,2,6.72,234622
C117311B72D20181122,5207066105228,1,1.35,117311
C117311B77D20181122,14459015561648,2,1.81,234622
...,...,...,...,...
C29501639B166D20190613,32030295799472,7,11.54,206511473
C29501639B44D20190625,31223305238023,6,11.77,177009834
C29501646B249D20190527,20804599415621,8,11.62,118006584


In [4]:
df_baskets2 = df_posdata.drop_duplicates(subset='Basket_ID', keep='first')
df_baskets2

,Basket_ID,Date,Barcode,Quantity,Value,LoyaltyCard_ID
0,C28002183B2D20181116,2018-11-16,2243124000000,1,5.74,28002183
6,C28098497B2D20190508,2019-05-08,5012501081155,1,2.36,28098497
10,C28325357B2D20181203,2018-12-03,5207066102197,1,2.39,28325357
17,C28325421B2D20190530,2019-05-30,5202178080234,1,1.08,28325421
23,C28325463B2D20181020,2018-10-20,5202178000331,1,2.38,28325463
...,...,...,...,...,...,...
298199,C28367985B818D20181015,2018-10-15,5201673005315,1,1.41,28367985
298201,C28337106B820D20181015,2018-10-15,5200327601651,1,1.19,28337106
298210,C28368049B822D20181015,2018-10-15,5201193100019,1,0.53,28368049
298214,C28368211B826D20181015,2018-10-15,5310115009795,1,0.92,28368211


In [5]:
df_baskets.reset_index()
df_custbask = pd.merge(df_baskets, df_baskets2, on='Basket_ID', how='inner')
df_custbask

,Basket_ID,Barcode_x,Quantity_x,Value_x,LoyaltyCard_ID_x,Date,Barcode_y,Quantity_y,Value_y,LoyaltyCard_ID_y
0,C117311B124D20181119,24889847425551,5,13.63,586555,2018-11-19,5201395645318,1,2.73,117311
1,C117311B140D20181027,36611051419462,8,23.17,938488,2018-10-27,4000508087486,1,4.69,117311
2,C117311B157D20181102,7454214154013,2,6.72,234622,2018-11-02,5214000154013,1,1.60,117311
3,C117311B72D20181122,5207066105228,1,1.35,117311,2018-11-22,5207066105228,1,1.35,117311
4,C117311B77D20181122,14459015561648,2,1.81,234622,2018-11-22,8718114724331,1,0.78,117311
...,...,...,...,...,...,...,...,...,...,...
48990,C29501639B166D20190613,32030295799472,7,11.54,206511473,2019-06-13,7622210688767,1,2.21,29501639
48991,C29501639B44D20190625,31223305238023,6,11.77,177009834,2019-06-25,5207066108809,1,0.36,29501639
48992,C29501646B249D20190527,20804599415621,8,11.62,118006584,2019-05-27,5201024513216,1,1.14,29501646
48993,C29522019B193D20190729,8588427830000,2,3.22,59044038,2019-07-29,3387390326468,1,1.78,29522019


In [6]:
df_custbask = df_custbask[['Basket_ID','Quantity_x','Value_x','LoyaltyCard_ID_y']]
df_custbask

,Basket_ID,Quantity_x,Value_x,LoyaltyCard_ID_y
0,C117311B124D20181119,5,13.63,117311
1,C117311B140D20181027,8,23.17,117311
2,C117311B157D20181102,2,6.72,117311
3,C117311B72D20181122,1,1.35,117311
4,C117311B77D20181122,2,1.81,117311
...,...,...,...,...
48990,C29501639B166D20190613,7,11.54,29501639
48991,C29501639B44D20190625,6,11.77,29501639
48992,C29501646B249D20190527,8,11.62,29501646
48993,C29522019B193D20190729,2,3.22,29522019


In [7]:
df_custbask.rename(columns = {'LoyaltyCard_ID_y':'LoyaltyCard_ID'},inplace='True')
df_baskpercust = df_custbask.groupby('LoyaltyCard_ID').count()
df_baskpercust

C:\Users\Λου\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Basket_ID,Quantity_x,Value_x
LoyaltyCard_ID,,,
117311,5,5,5
122293,2,2,2
28002183,81,81,81
28011966,5,5,5
28012303,2,2,2
...,...,...,...
29488866,1,1,1
29501624,1,1,1
29501639,2,2,2


In [8]:
df_baskpercust.rename(columns = {'Basket_ID':'Number of Baskets'}, inplace = True)
df_baskpercust = df_baskpercust.drop(['Quantity_x', 'Value_x'], axis=1)
df_baskpercust

,Number of Baskets
LoyaltyCard_ID,
117311,5
122293,2
28002183,81
28011966,5
28012303,2
...,...
29488866,1
29501624,1
29501639,2


In [9]:
df_baskpercust2 = df_custbask.groupby('LoyaltyCard_ID').mean()
df_baskpercust2

,Quantity_x,Value_x
LoyaltyCard_ID,,
117311,3.600000,9.336000
122293,12.500000,20.265000
28002183,3.222222,8.210988
28011966,6.000000,9.280000
28012303,5.500000,21.365000
...,...,...
29488866,1.000000,17.990000
29501624,3.000000,6.050000
29501639,6.500000,11.655000


In [10]:
df_baskpercust2.rename(columns = {'Quantity_x':'Average Quantity', 'Value_x':'Average Value'}, inplace = True)

In [12]:
df_baskpercust2

,Average Quantity,Average Value
LoyaltyCard_ID,,
117311,3.600000,9.336000
122293,12.500000,20.265000
28002183,3.222222,8.210988
28011966,6.000000,9.280000
28012303,5.500000,21.365000
...,...,...
29488866,1.000000,17.990000
29501624,3.000000,6.050000
29501639,6.500000,11.655000


In [13]:
df_baskpercustf = df_baskpercust.reset_index()
df_baskpercustf

,LoyaltyCard_ID,Number of Baskets
0,117311,5
1,122293,2
2,28002183,81
3,28011966,5
4,28012303,2
...,...,...
2401,29488866,1
2402,29501624,1
2403,29501639,2
2404,29501646,1


In [14]:
df_baskpercust2f = df_baskpercust2.reset_index()
df_baskpercust2f

,LoyaltyCard_ID,Average Quantity,Average Value
0,117311,3.600000,9.336000
1,122293,12.500000,20.265000
2,28002183,3.222222,8.210988
3,28011966,6.000000,9.280000
4,28012303,5.500000,21.365000
...,...,...,...
2401,29488866,1.000000,17.990000
2402,29501624,3.000000,6.050000
2403,29501639,6.500000,11.655000
2404,29501646,8.000000,11.620000


In [15]:
df_custStats = pd.merge(df_baskpercust2f, df_baskpercustf, on = 'LoyaltyCard_ID', how='inner')
df_custStats

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets
0,117311,3.600000,9.336000,5
1,122293,12.500000,20.265000,2
2,28002183,3.222222,8.210988,81
3,28011966,6.000000,9.280000,5
4,28012303,5.500000,21.365000,2
...,...,...,...,...
2401,29488866,1.000000,17.990000,1
2402,29501624,3.000000,6.050000,1
2403,29501639,6.500000,11.655000,2
2404,29501646,8.000000,11.620000,1


In [16]:
r = df_posdata.groupby('Basket_ID').sum('Quantity')
r.reset_index()
r = r.drop(['Barcode','LoyaltyCard_ID'], axis=1)
r.reset_index()

,Basket_ID,Quantity,Value
0,C117311B124D20181119,5,13.63
1,C117311B140D20181027,8,23.17
2,C117311B157D20181102,2,6.72
3,C117311B72D20181122,1,1.35
4,C117311B77D20181122,2,1.81
...,...,...,...
48990,C29501639B166D20190613,7,11.54
48991,C29501639B44D20190625,6,11.77
48992,C29501646B249D20190527,8,11.62
48993,C29522019B193D20190729,2,3.22


In [17]:
pos = df_posdata[['Basket_ID','Date','LoyaltyCard_ID']]
pos

,Basket_ID,Date,LoyaltyCard_ID
0,C28002183B2D20181116,2018-11-16,28002183
1,C28002183B2D20181116,2018-11-16,28002183
2,C28002183B2D20181116,2018-11-16,28002183
3,C28002183B2D20181116,2018-11-16,28002183
4,C28002183B2D20181116,2018-11-16,28002183
...,...,...,...
298216,C28368211B826D20181015,2018-10-15,28368211
298217,C28368211B826D20181015,2018-10-15,28368211
298218,C28368211B826D20181015,2018-10-15,28368211
298219,C29081476B841D20181015,2018-10-15,29081476


In [18]:
r = pd.merge(r,pos, on='Basket_ID', how='inner')
r = r.drop_duplicates('Basket_ID')
r

,Basket_ID,Quantity,Value,Date,LoyaltyCard_ID
0,C117311B124D20181119,5,13.63,2018-11-19,117311
5,C117311B140D20181027,8,23.17,2018-10-27,117311
13,C117311B157D20181102,2,6.72,2018-11-02,117311
15,C117311B72D20181122,1,1.35,2018-11-22,117311
16,C117311B77D20181122,2,1.81,2018-11-22,117311
...,...,...,...,...,...
298199,C29501639B166D20190613,7,11.54,2019-06-13,29501639
298206,C29501639B44D20190625,6,11.77,2019-06-25,29501639
298212,C29501646B249D20190527,8,11.62,2019-05-27,29501646
298216,C29522019B193D20190729,2,3.22,2019-07-29,29522019


In [19]:
r['Month'] = r['Date'].dt.month 
re = r.reset_index(drop=True)
re

,Basket_ID,Quantity,Value,Date,LoyaltyCard_ID,Month
0,C117311B124D20181119,5,13.63,2018-11-19,117311,11
1,C117311B140D20181027,8,23.17,2018-10-27,117311,10
2,C117311B157D20181102,2,6.72,2018-11-02,117311,11
3,C117311B72D20181122,1,1.35,2018-11-22,117311,11
4,C117311B77D20181122,2,1.81,2018-11-22,117311,11
...,...,...,...,...,...,...
48990,C29501639B166D20190613,7,11.54,2019-06-13,29501639,6
48991,C29501639B44D20190625,6,11.77,2019-06-25,29501639,6
48992,C29501646B249D20190527,8,11.62,2019-05-27,29501646,5
48993,C29522019B193D20190729,2,3.22,2019-07-29,29522019,7


In [20]:
xv = re.groupby('LoyaltyCard_ID').count()
yvalues = [1,2,3,4,5,6,7,8,9,10,11,12]
xvalues = xv.index
freq = pd.DataFrame(0, index=xvalues, columns=yvalues)
freq

,1,2,3,4,5,6,7,8,9,10,11,12
LoyaltyCard_ID,,,,,,,,,,,,
117311,0,0,0,0,0,0,0,0,0,0,0,0
122293,0,0,0,0,0,0,0,0,0,0,0,0
28002183,0,0,0,0,0,0,0,0,0,0,0,0
28011966,0,0,0,0,0,0,0,0,0,0,0,0
28012303,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
29488866,0,0,0,0,0,0,0,0,0,0,0,0
29501624,0,0,0,0,0,0,0,0,0,0,0,0
29501639,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
#ΑΡΧΙΚΟΠΟΙΗΣΗ ΠΙΝΑΚΑ ΓΙΑ ΕΡΩΤΗΜΑ 3
for i in range(len(re)):
    freq.at[(re.loc[i,"LoyaltyCard_ID"]),(re.loc[i,"Month"])] +=1

In [22]:
freq

,1,2,3,4,5,6,7,8,9,10,11,12
LoyaltyCard_ID,,,,,,,,,,,,
117311,0,0,0,0,0,0,0,0,0,1,4,0
122293,0,2,0,0,0,0,0,0,0,0,0,0
28002183,6,11,6,8,10,7,0,0,5,9,8,11
28011966,0,0,0,0,0,0,0,0,4,1,0,0
28012303,0,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
29488866,0,0,1,0,0,0,0,0,0,0,0,0
29501624,0,0,0,0,0,0,0,1,0,0,0,0
29501639,0,0,0,0,0,2,0,0,0,0,0,0


In [40]:
freq['totalmonths'] = 0
freq

,LoyaltyCard_ID,1,2,3,4,5,6,7,8,9,10,11,12,totalmonths
0,117311,0,0,0,0,0,0,0,0,0,1,4,0,0
1,122293,0,2,0,0,0,0,0,0,0,0,0,0,0
2,28002183,6,11,6,8,10,7,0,0,5,9,8,11,0
3,28011966,0,0,0,0,0,0,0,0,4,1,0,0,0
4,28012303,0,0,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,29488866,0,0,1,0,0,0,0,0,0,0,0,0,0
2402,29501624,0,0,0,0,0,0,0,1,0,0,0,0,0
2403,29501639,0,0,0,0,0,2,0,0,0,0,0,0,0
2404,29501646,0,0,0,0,1,0,0,0,0,0,0,0,0


In [41]:
for i in range (len(freq)): 
    for j in range(1, 13):
        if (freq.at[i,j] != 0):
            freq.at[i,'totalmonths'] +=1
            
freq


,LoyaltyCard_ID,1,2,3,4,5,6,7,8,9,10,11,12,totalmonths
0,117311,0,0,0,0,0,0,0,0,0,1,4,0,2
1,122293,0,2,0,0,0,0,0,0,0,0,0,0,1
2,28002183,6,11,6,8,10,7,0,0,5,9,8,11,10
3,28011966,0,0,0,0,0,0,0,0,4,1,0,0,2
4,28012303,0,0,0,0,0,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,29488866,0,0,1,0,0,0,0,0,0,0,0,0,1
2402,29501624,0,0,0,0,0,0,0,1,0,0,0,0,1
2403,29501639,0,0,0,0,0,2,0,0,0,0,0,0,1
2404,29501646,0,0,0,0,1,0,0,0,0,0,0,0,1


In [43]:
freq.loc[(freq['totalmonths'] >=7)]

,LoyaltyCard_ID,1,2,3,4,5,6,7,8,9,10,11,12,totalmonths
2,28002183,6,11,6,8,10,7,0,0,5,9,8,11,10
12,28030043,1,1,2,1,0,0,0,0,2,0,1,2,7
14,28030119,4,1,1,1,0,2,1,0,2,2,1,6,10
18,28031107,3,2,3,1,0,0,0,0,1,0,1,2,7
25,28036041,4,3,2,0,0,0,0,0,2,5,5,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,29359163,0,12,17,17,26,16,11,26,0,0,0,0,7
2383,29359164,0,18,35,37,27,38,45,17,0,0,0,0,7
2384,29359165,0,7,17,18,9,17,19,11,0,0,0,0,7
2385,29359166,0,15,25,28,24,23,31,25,0,0,0,0,7


In [23]:
df_custStats

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets
0,117311,3.600000,9.336000,5
1,122293,12.500000,20.265000,2
2,28002183,3.222222,8.210988,81
3,28011966,6.000000,9.280000,5
4,28012303,5.500000,21.365000,2
...,...,...,...,...
2401,29488866,1.000000,17.990000,1
2402,29501624,3.000000,6.050000,1
2403,29501639,6.500000,11.655000,2
2404,29501646,8.000000,11.620000,1


In [44]:
customerstats = pd.merge(df_custStats, freq, on='LoyaltyCard_ID', how='inner')
customerstats

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets,1,2,3,4,5,6,7,8,9,10,11,12,totalmonths
0,117311,3.600000,9.336000,5,0,0,0,0,0,0,0,0,0,1,4,0,2
1,122293,12.500000,20.265000,2,0,2,0,0,0,0,0,0,0,0,0,0,1
2,28002183,3.222222,8.210988,81,6,11,6,8,10,7,0,0,5,9,8,11,10
3,28011966,6.000000,9.280000,5,0,0,0,0,0,0,0,0,4,1,0,0,2
4,28012303,5.500000,21.365000,2,0,0,0,0,0,1,0,0,0,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,29488866,1.000000,17.990000,1,0,0,1,0,0,0,0,0,0,0,0,0,1
2402,29501624,3.000000,6.050000,1,0,0,0,0,0,0,0,1,0,0,0,0,1
2403,29501639,6.500000,11.655000,2,0,0,0,0,0,2,0,0,0,0,0,0,1
2404,29501646,8.000000,11.620000,1,0,0,0,0,1,0,0,0,0,0,0,0,1


In [46]:
customerstats = customerstats[['LoyaltyCard_ID','Average Quantity','Average Value','Number of Baskets','totalmonths']]
customerstats

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets,totalmonths
0,117311,3.600000,9.336000,5,2
1,122293,12.500000,20.265000,2,1
2,28002183,3.222222,8.210988,81,10
3,28011966,6.000000,9.280000,5,2
4,28012303,5.500000,21.365000,2,2
...,...,...,...,...,...
2401,29488866,1.000000,17.990000,1,1
2402,29501624,3.000000,6.050000,1,1
2403,29501639,6.500000,11.655000,2,1
2404,29501646,8.000000,11.620000,1,1


In [51]:
loyals = customerstats.loc[(customerstats['totalmonths'] >= 8) & (customerstats['Number of Baskets'] >=16)]
loyals

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets,totalmonths
2,28002183,3.222222,8.210988,81,10
14,28030119,2.666667,10.132381,21,10
29,28098497,6.233333,12.851500,60,12
32,28107493,8.041667,19.515833,24,10
39,28214782,8.204082,17.171633,49,12
...,...,...,...,...,...
2194,29338188,7.507463,8.970000,67,8
2195,29338189,4.925926,10.502407,108,8
2199,29338193,7.600000,20.929750,40,8
2200,29338194,15.201613,38.626290,124,8


In [54]:
premium = loyals.loc[(loyals['Average Value'] > 15)]
premium

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets,totalmonths
32,28107493,8.041667,19.515833,24,10
39,28214782,8.204082,17.171633,49,12
73,28325306,9.722222,17.840556,18,10
75,28325310,11.219512,25.854634,41,12
77,28325314,6.882353,15.646765,34,12
...,...,...,...,...,...
2165,29338159,7.984127,19.371905,63,8
2172,29338166,9.951220,20.986341,41,8
2179,29338173,7.302326,16.105814,43,8
2199,29338193,7.600000,20.929750,40,8


In [52]:
often = customerstats.loc[(customerstats['totalmonths'] >= 4) & (customerstats['totalmonths'] <= 7) & (customerstats['Number of Baskets'] >=10)]
often

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets,totalmonths
12,28030043,3.300000,7.934000,10,7
18,28031107,7.846154,23.849231,13,7
24,28034685,8.900000,18.845667,30,5
25,28036041,9.260870,19.529565,23,7
27,28036576,3.571429,10.547143,14,7
...,...,...,...,...,...
2384,29359165,5.285714,9.973571,98,7
2385,29359166,3.766082,6.729123,171,7
2386,29359168,4.545455,9.025758,33,6
2387,29359169,4.171642,7.079328,134,7


In [61]:
lampros = pd.merge(df_posdata, df_a, on='Basket_ID', how='inner')
lampros

,Basket_ID,Date,Barcode_x,Quantity_x,Value_x,LoyaltyCard_ID_x,Grouped,Barcode_y,Quantity_y,Value_y,LoyaltyCard_ID_y
0,C28002183B2D20181116,2018-11-16,2243124000000,1,5.74,28002183,"ΤΥΡΟΚΟΜΙΚΑ,ΑΛΛΑΝΤΙΚΑ,ΚΡΟΥΑΣΑΝ,ΤΥΡΟΚΟΜΙΚΑ,ΚΡΟΥΑ...",22333370476772,6,13.48,168013098
1,C28002183B2D20181116,2018-11-16,2242827000000,1,1.40,28002183,"ΤΥΡΟΚΟΜΙΚΑ,ΑΛΛΑΝΤΙΚΑ,ΚΡΟΥΑΣΑΝ,ΤΥΡΟΚΟΜΙΚΑ,ΚΡΟΥΑ...",22333370476772,6,13.48,168013098
2,C28002183B2D20181116,2018-11-16,5204416014100,1,1.09,28002183,"ΤΥΡΟΚΟΜΙΚΑ,ΑΛΛΑΝΤΙΚΑ,ΚΡΟΥΑΣΑΝ,ΤΥΡΟΚΟΜΙΚΑ,ΚΡΟΥΑ...",22333370476772,6,13.48,168013098
3,C28002183B2D20181116,2018-11-16,2240547000000,1,4.58,28002183,"ΤΥΡΟΚΟΜΙΚΑ,ΑΛΛΑΝΤΙΚΑ,ΚΡΟΥΑΣΑΝ,ΤΥΡΟΚΟΜΙΚΑ,ΚΡΟΥΑ...",22333370476772,6,13.48,168013098
4,C28002183B2D20181116,2018-11-16,5201228231336,1,0.34,28002183,"ΤΥΡΟΚΟΜΙΚΑ,ΑΛΛΑΝΤΙΚΑ,ΚΡΟΥΑΣΑΝ,ΤΥΡΟΚΟΜΙΚΑ,ΚΡΟΥΑ...",22333370476772,6,13.48,168013098
...,...,...,...,...,...,...,...,...,...,...,...
293788,C28368211B826D20181015,2018-10-15,5449000214911,1,0.60,28368211,"ΓΑΛΑΤΑ,ΨΩΜΙ,COLA,ΑΝΑΨΥΚΤΙΚΑ,WAFER",26376535397114,5,3.76,141841055
293789,C28368211B826D20181015,2018-10-15,5201037506878,1,0.82,28368211,"ΓΑΛΑΤΑ,ΨΩΜΙ,COLA,ΑΝΑΨΥΚΤΙΚΑ,WAFER",26376535397114,5,3.76,141841055
293790,C28368211B826D20181015,2018-10-15,5214000154129,1,0.98,28368211,"ΓΑΛΑΤΑ,ΨΩΜΙ,COLA,ΑΝΑΨΥΚΤΙΚΑ,WAFER",26376535397114,5,3.76,141841055
293791,C29081476B841D20181015,2018-10-15,5202178001116,1,0.85,29081476,"ΓΑΛΑΤΑ,ΚΡΟΥΑΣΑΝ",10403186617698,2,2.74,58162952


In [62]:
lampros = lampros.drop_duplicates(subset='Basket_ID', keep='first')
lampros

,Basket_ID,Date,Barcode_x,Quantity_x,Value_x,LoyaltyCard_ID_x,Grouped,Barcode_y,Quantity_y,Value_y,LoyaltyCard_ID_y
0,C28002183B2D20181116,2018-11-16,2243124000000,1,5.74,28002183,"ΤΥΡΟΚΟΜΙΚΑ,ΑΛΛΑΝΤΙΚΑ,ΚΡΟΥΑΣΑΝ,ΤΥΡΟΚΟΜΙΚΑ,ΚΡΟΥΑ...",22333370476772,6,13.48,168013098
6,C28098497B2D20190508,2019-05-08,5012501081155,1,2.36,28098497,"ΚΡΟΥΑΣΑΝ,ΣΟΚΟΛΑΤΕΣ,WAFER,ΜΠΙΣΚΟΤΑ",20615874247556,4,5.72,112393988
10,C28325357B2D20181203,2018-12-03,5207066102197,1,2.39,28325357,"ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,ΠΑΝΕΣ ΑΚΡΑΤΕΙΑΣ,ΖΑΧΑΡΩΔΗ,ΠΕΡΙΠ...",28932545090947,7,13.94,198277499
17,C28325421B2D20190530,2019-05-30,5202178080234,1,1.08,28325421,"ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,ΓΙΑΟΥΡΤΙΑ,ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,ΦΡΥΓ...",30842068217376,6,12.20,169952526
23,C28325463B2D20181020,2018-10-20,5202178000331,1,2.38,28325463,"ΤΥΡΟΚΟΜΙΚΑ,ΓΑΛΑΤΑ,ΜΠΙΣΚΟΤΑ,ΤΥΡΟΚΟΜΙΚΑ",20809103150766,4,7.08,113301852
...,...,...,...,...,...,...,...,...,...,...,...
293771,C28367985B818D20181015,2018-10-15,5201673005315,1,1.41,28367985,"ΨΩΜΙ,ΚΡΟΥΑΣΑΝ",10402681621897,2,3.30,56735970
293773,C28337106B820D20181015,2018-10-15,5200327601651,1,1.19,28337106,"ΤΥΡΟΚΟΜΙΚΑ,ΜΠΙΣΚΟΤΑ,ΓΙΑΟΥΡΤΙΑ,ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,...",40896433351007,9,18.39,255033954
293782,C28368049B822D20181015,2018-10-15,5201193100019,1,0.53,28368049,"ΤΥΡΟΚΟΜΙΚΑ,ΖΥΜΑΡΙΚΑ,ΤΥΡΟΚΟΜΙΚΑ,ΤΟΜΑΤΙΚΑ",14884299600037,4,3.89,113472196
293786,C28368211B826D20181015,2018-10-15,5310115009795,1,0.92,28368211,"ΓΑΛΑΤΑ,ΨΩΜΙ,COLA,ΑΝΑΨΥΚΤΙΚΑ,WAFER",26376535397114,5,3.76,141841055


In [66]:
mergeme = lampros[['Basket_ID','Grouped','LoyaltyCard_ID']]
mergeme = mergeme.reset_index(drop = True)
mergeme

,Basket_ID,Grouped,LoyaltyCard_ID
0,C28002183B2D20181116,"ΤΥΡΟΚΟΜΙΚΑ,ΑΛΛΑΝΤΙΚΑ,ΚΡΟΥΑΣΑΝ,ΤΥΡΟΚΟΜΙΚΑ,ΚΡΟΥΑ...",28002183
1,C28098497B2D20190508,"ΚΡΟΥΑΣΑΝ,ΣΟΚΟΛΑΤΕΣ,WAFER,ΜΠΙΣΚΟΤΑ",28098497
2,C28325357B2D20181203,"ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,ΠΑΝΕΣ ΑΚΡΑΤΕΙΑΣ,ΖΑΧΑΡΩΔΗ,ΠΕΡΙΠ...",28325357
3,C28325421B2D20190530,"ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,ΓΙΑΟΥΡΤΙΑ,ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,ΦΡΥΓ...",28325421
4,C28325463B2D20181020,"ΤΥΡΟΚΟΜΙΚΑ,ΓΑΛΑΤΑ,ΜΠΙΣΚΟΤΑ,ΤΥΡΟΚΟΜΙΚΑ",28325463
...,...,...,...
45031,C28367985B818D20181015,"ΨΩΜΙ,ΚΡΟΥΑΣΑΝ",28367985
45032,C28337106B820D20181015,"ΤΥΡΟΚΟΜΙΚΑ,ΜΠΙΣΚΟΤΑ,ΓΙΑΟΥΡΤΙΑ,ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,...",28337106
45033,C28368049B822D20181015,"ΤΥΡΟΚΟΜΙΚΑ,ΖΥΜΑΡΙΚΑ,ΤΥΡΟΚΟΜΙΚΑ,ΤΟΜΑΤΙΚΑ",28368049
45034,C28368211B826D20181015,"ΓΑΛΑΤΑ,ΨΩΜΙ,COLA,ΑΝΑΨΥΚΤΙΚΑ,WAFER",28368211


In [63]:
lampros.rename(columns = {'LoyaltyCard_ID_x':'LoyaltyCard_ID'},inplace='True')

C:\Users\chris\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [68]:
oftenbaskets = pd.merge(often, mergeme, on='LoyaltyCard_ID', how='inner')
oftenbaskets

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets,totalmonths,Basket_ID,Grouped
0,28030043,3.30,7.934000,10,7,C28030043B95D20190119,"ΤΥΡΟΚΟΜΙΚΑ,ΤΥΡΟΚΟΜΙΚΑ,ΔΗΜΗΤΡΙΑΚΑ,DRESSINGS"
1,28030043,3.30,7.934000,10,7,C28030043B130D20181207,"ΤΥΡΟΚΟΜΙΚΑ,ΤΥΡΟΚΟΜΙΚΑ,ΣΟΚΟΛΑΤΕΣ,ΚΡΟΥΑΣΑΝ,ΤΣΟΥΡΕΚΙ"
2,28030043,3.30,7.934000,10,7,C28030043B249D20190207,"ΓΑΛΑΤΑ,ΠΟΥΡΕΣ"
3,28030043,3.30,7.934000,10,7,C28030043B476D20190315,"ΜΠΙΣΚΟΤΑ,ΛΑΔΙ,ΞΗΡΟΙ ΚΑΡΠΟΙ/ΣΝΑΚΣ,ΖΑΧΑΡΗ"
4,28030043,3.30,7.934000,10,7,C28030043B498D20190320,"ΨΩΜΙ,ΤΥΡΟΚΟΜΙΚΑ,ΛΑΔΙ"
...,...,...,...,...,...,...,...
10523,29438682,7.75,14.368333,12,4,C29438682B226D20190422,"ΤΥΡΟΚΟΜΙΚΑ,ΨΩΜΙ,ΧΑΛΒΑΔΕΣ ΤΑΧΙΝΙ,ΒΡΕΦΙΚΗ ΤΡΟΦΗ,..."
10524,29438682,7.75,14.368333,12,4,C29438682B226D20190511,"ΤΥΡΟΚΟΜΙΚΑ,ΧΥΜΟΙ,ΨΩΜΙ,ΖΑΧΑΡΗ,ΑΛΕΥΡΙ,ΕΙΔΗ ΜΙΑΣ ..."
10525,29438682,7.75,14.368333,12,4,C29438682B403D20190529,"ΑΠΟΡΡΥΠΑΝΤΙΚΑ,ΓΑΛΑΤΑ,ΕΙΔΗ ΜΙΑΣ ΧΡΗΣΗΣ,ΕΙΔΗ ΜΙΑ..."
10526,29438682,7.75,14.368333,12,4,C29438682B458D20190515,"ΤΥΡΟΚΟΜΙΚΑ,ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ,ΚΡΟΥΑΣΑΝ,ΒΡΕΦΙΚΗ ΤΡΟΦΗ"


In [70]:

# Referenced: https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary
from itertools import combinations
from collections import Counter

count = Counter()

for row in oftenbaskets['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key,value in count.most_common(30):
    print(key, value)


('ΤΥΡΟΚΟΜΙΚΑ', 'ΤΥΡΟΚΟΜΙΚΑ') 3847
('ΤΥΡΟΚΟΜΙΚΑ', 'ΑΛΛΑΝΤΙΚΑ') 3217
('ΤΥΡΟΚΟΜΙΚΑ', 'ΓΑΛΑΤΑ') 2140
('ΤΥΡΟΚΟΜΙΚΑ', 'ΨΩΜΙ') 1881
('ΤΥΡΟΚΟΜΙΚΑ', 'ΓΙΑΟΥΡΤΙΑ') 1651
('ΨΩΜΙ', 'ΤΥΡΟΚΟΜΙΚΑ') 1562
('ΑΛΛΑΝΤΙΚΑ', 'ΤΥΡΟΚΟΜΙΚΑ') 1467
('ΤΥΡΟΚΟΜΙΚΑ', 'ΖΥΜΑΡΙΚΑ') 1400
('ΤΥΡΟΚΟΜΙΚΑ', 'ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ') 1320
('ΤΥΡΟΚΟΜΙΚΑ', 'ΑΠΟΡΡΥΠΑΝΤΙΚΑ') 1229
('ΤΥΡΟΚΟΜΙΚΑ', 'ΣΟΚΟΛΑΤΕΣ') 1212
('ΤΥΡΟΚΟΜΙΚΑ', 'ΞΗΡΟΙ ΚΑΡΠΟΙ/ΣΝΑΚΣ') 1164
('ΤΥΡΟΚΟΜΙΚΑ', 'ΜΠΙΣΚΟΤΑ') 1160
('ΨΩΜΙ', 'ΑΛΛΑΝΤΙΚΑ') 1159
('ΤΥΡΟΚΟΜΙΚΑ', 'ΚΡΕΑΤΙΚΑ/ΨΑΡΙΚΑ') 1138
('ΨΩΜΙ', 'ΓΑΛΑΤΑ') 1079
('ΤΥΡΟΚΟΜΙΚΑ', 'ΚΑΦΕΣ') 1044
('ΤΥΡΟΚΟΜΙΚΑ', 'ΚΑΘΑΡΙΣΤΙΚΑ') 985
('ΞΗΡΟΙ ΚΑΡΠΟΙ/ΣΝΑΚΣ', 'ΞΗΡΟΙ ΚΑΡΠΟΙ/ΣΝΑΚΣ') 899
('ΤΥΡΟΚΟΜΙΚΑ', 'COLA') 854
('ΣΟΚΟΛΑΤΕΣ', 'ΣΟΚΟΛΑΤΕΣ') 837
('ΤΥΡΟΚΟΜΙΚΑ', 'ΒΟΥΤΥΡΑ') 825
('ΨΩΜΙ', 'ΓΙΑΟΥΡΤΙΑ') 767
('ΠΡΟΙΟΝΤΑ ΧΑΡΤΟΥ', 'ΤΥΡΟΚΟΜΙΚΑ') 754
('ΤΥΡΟΚΟΜΙΚΑ', 'ΤΟΜΑΤΙΚΑ') 711
('ΑΛΛΑΝΤΙΚΑ', 'ΓΑΛΑΤΑ') 709
('ΓΑΛΑΤΑ', 'ΓΙΑΟΥΡΤΙΑ') 678
('ΨΩΜΙ', 'ΨΩΜΙ') 676
('ΤΥΡΟΚΟΜΙΚΑ', 'ΧΥΜΟΙ') 659
('ΤΥΡΟΚΟΜΙΚΑ', 'ΑΝΑΨΥΚΤΙΚΑ') 649


In [24]:
df_loyal = df_custStats.loc[(df_custStats['Average Quantity'] >= 3) & (df_custStats['Average Value'] >= 6) & (df_custStats['Number of Baskets'] >= 8)]
df_loyal

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets
2,28002183,3.222222,8.210988,81
12,28030043,3.300000,7.934000,10
18,28031107,7.846154,23.849231,13
20,28031607,15.333333,28.542222,9
24,28034685,8.900000,18.845667,30
...,...,...,...,...
2385,29359166,3.766082,6.729123,171
2386,29359168,4.545455,9.025758,33
2387,29359169,4.171642,7.079328,134
2392,29415975,5.500000,11.945000,8


In [25]:
df_loyal = df_custStats.loc[(df_custStats['Average Value'] >= 10) & (df_custStats['Number of Baskets'] >= 15)]
df_loyal

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets
14,28030119,2.666667,10.132381,21
24,28034685,8.900000,18.845667,30
25,28036041,9.260870,19.529565,23
29,28098497,6.233333,12.851500,60
32,28107493,8.041667,19.515833,24
...,...,...,...,...
2350,29338350,9.333333,17.057143,21
2351,29338351,11.000000,18.432778,18
2357,29338357,8.470588,14.887647,17
2360,29338362,11.480000,27.265200,25


In [26]:
df_loyal = df_custStats.loc[(df_custStats['Number of Baskets'] >= 15)]
df_loyal

,LoyaltyCard_ID,Average Quantity,Average Value,Number of Baskets
2,28002183,3.222222,8.210988,81
14,28030119,2.666667,10.132381,21
24,28034685,8.900000,18.845667,30
25,28036041,9.260870,19.529565,23
29,28098497,6.233333,12.851500,60
...,...,...,...,...
2383,29359164,4.281106,6.416544,217
2384,29359165,5.285714,9.973571,98
2385,29359166,3.766082,6.729123,171
2386,29359168,4.545455,9.025758,33
